In [2]:
import pandas as pd
import numpy as np
import findspark
findspark.init()

In [3]:
import pyspark

create pyspark context

In [4]:
sc=pyspark.SparkContext(appName='joao')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/02 20:41:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


resilient distributed dataset

In [40]:
rdd=sc.parallelize([i for i in range(10)])
rdd_squared=rdd.map(lambda x: x**2)

In [41]:
rdd_squared.collect()

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

## DataFrame

In [57]:
from sklearn.datasets import load_breast_cancer

In [20]:
spark_df=pyspark.sql.SparkSession(sc)

In [58]:
d=load_breast_cancer()
df=pd.DataFrame(d['data'])
df['target']=d['target']

In [59]:
df=spark_df.createDataFrame(df.to_records().tolist())

In [60]:
df.show(3)

22/07/02 22:16:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+---+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+---+
| _1|   _2|   _3|   _4|    _5|     _6|     _7|    _8|     _9|   _10|    _11|   _12|   _13|  _14|  _15|     _16|    _17|    _18|    _19|    _20|     _21|  _22|  _23|  _24|   _25|   _26|   _27|   _28|   _29|   _30|    _31|_32|
+---+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+---+
|  0|17.99|10.38|122.8|1001.0| 0.1184| 0.2776|0.3001| 0.1471|0.2419|0.07871| 1.095|0.9053|8.589|153.4|0.006399|0.04904|0.05373|0.01587|0.03003|0.

## Machine Learning

In [53]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint

In [61]:
data=[]
for row, y in zip(d['data'],d['target']):
    p=LabeledPoint(y,row)
    data.append(p)

In [62]:
data=sc.parallelize(data)

In [63]:
d.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [64]:
lrm=LogisticRegressionWithSGD.train(data,iterations=100)

/Users/JoaoGomes/miniconda3/envs/spark/lib/python3.9/site-packages/pyspark/mllib/classification.py:395: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  warnings.warn(


In [66]:
lrm.weights

DenseVector([21.4597, 39.6495, 131.1845, 169.2285, 0.2244, 0.073, -0.1287, -0.0651, 0.4243, 0.1678, 0.1258, 3.1275, 0.6994, -43.6572, 0.0191, 0.0305, 0.0299, 0.0138, 0.0525, 0.0083, 20.6254, 50.6104, 125.4718, -166.4565, 0.2939, 0.0845, -0.1652, -0.0265, 0.6148, 0.1871])